## Makes Folders 

In [4]:
import os

# Base directory
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"

# Add Players Names based on who you need to add or take out
player_names = [
    "Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
    "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
    "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
    "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
    "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
    "Zach Liggett", "Ryan Buckner"
]

# Create directories for each player
for name in player_names:
    os.makedirs(os.path.join(base_dir, name), exist_ok=True)

print("Directories created for all players!")


Directories created for all players!


## Progress Graph

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# # Add Players Names based on who you need to add or take out, This is based off the sheets in excel
player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Ryan Buckner"]

# Base directory
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
start_date = pd.Timestamp('2023-10-23')

def get_metric_initials(metric):
    """Get metric initials from metric name."""
    words = metric.split()
    initials = ''.join([word[0].upper() for word in words if word not in ('(', ')', ' ')]).replace("(","").replace(")","")
    return initials

def plot_metric(df, metric_column, player_initials, ax):
    """Plot specified metric from dataframe."""
    
    metric_initials = get_metric_initials(metric_column)
    
    # Group by custom week number and compute the average of the metric
    avg_metric_per_week = df.groupby('Week')[metric_column].mean()

    # Plot the data
    avg_metric_per_week.plot(ax=ax, linestyle='-', marker='o', markersize=4)

    # Add data labels with positioning adjustments
    y_min, y_max = ax.get_ylim()
    offset_factor = (y_max - y_min) * 0.05  # 5% of y-axis range
    
    for x, y in avg_metric_per_week.items():
        if y > (y_max + y_min) / 2:
            vertical_alignment = 'top'
            offset = -offset_factor
        else:
            vertical_alignment = 'bottom'
            offset = offset_factor
            
        ax.text(x, y + offset, f"{y:.2f}", ha='center', va=vertical_alignment, fontsize=7)

    # Set the title and labels with reduced font size
    ax.set_title(f"{player_initials} {metric_initials}", fontsize=10)
    ax.set_xlabel("Week", fontsize=9)
    ax.set_ylabel(metric_initials, fontsize=9)
    ax.tick_params(axis='both', which='major', labelsize=8)

    # Adjust the graph size based on the data
    ax.set_xlim(0.5, df['Week'].max() + 0.5)

    # Add tick marks on the x-axis for each week
    ax.set_xticks(range(1, df['Week'].max() + 1))

    # Adjust aesthetics
    ax.grid(True)
    ax.set_aspect(1.0/ax.get_data_ratio()*2.5/2.5)  # Set the aspect to make graph 2.5x2.5

def generate_and_save_graphs_for_player(player_name):
    # Load player data
    df_player = pd.read_excel(f"{base_dir}/Fall2Winter.xlsx", sheet_name=player_name, skiprows=8, engine='openpyxl')
    
    # Convert the 'Date' column to datetime format
    df_player['Date'] = pd.to_datetime(df_player['Date'])

    # Calculate the week number based on the start date
    df_player['Week'] = ((df_player['Date'] - start_date).dt.days // 7) + 1

    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(1, 3, figsize=(9, 3))

    # Player initials
    player_initials = ''.join([name[0] for name in player_name.split()])

    # Generate plots
    plot_metric(df_player, 'Bat Speed (mph)', player_initials, axs[0])
    plot_metric(df_player, 'Rotational Acceleration (g)', player_initials, axs[1])
    plot_metric(df_player, 'Attack Angle (deg)', player_initials, axs[2])

    # Adjust layout
    plt.tight_layout()

    # Save the plot as an image
    output_path = f"{base_dir}/{player_name}/{player_name} Blast Progress.png"
    plt.savefig(output_path, format='png', dpi=300)
    
    # Close the plot to free up memory
    plt.close()

# Generate and save graphs for each player
for player in player_names:
    generate_and_save_graphs_for_player(player)

print("Graphs generated and saved for all players!")


Graphs generated and saved for all players!


## Game vs Practice Radar Chart 

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Add Players Names based on who you need to add or take out

# Change the name of the excel files to match... look at the text for xlsx documents
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
week5_path = f"{base_dir}/Fall2Winter.xlsx"
game_swings_path = f"{base_dir}/Game Swings.xlsx"

player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Zach Liggett", "Ryan Buckner"]

# Metrics list
metrics = ["Plane Score", "Connection Score", "Rotation Score", "Bat Speed (mph)", 
           "Rotational Acceleration (g)", "On Plane Efficiency (%)", "Attack Angle (deg)", 
           "Early Connection (deg)", "Connection at Impact (deg)", "Vertical Bat Angle (deg)", 
           "Power (kW)", "Time to Contact (sec)", "Peak Hand Speed (mph)"]

for player in player_names:
    player_data = pd.read_excel(week5_path, sheet_name=player, skiprows=8, engine='openpyxl')
    
    # Calculate the mean and standard deviation for each metric
    means = [round(player_data[metric].mean(), 3) for metric in metrics]
    std_devs = [round(player_data[metric].std(), 3) for metric in metrics]
    
    # Create a table using matplotlib
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=[means, std_devs], 
                     colLabels=metrics, 
                     rowLabels=['Average', 'Standard Deviation'], 
                     cellLoc = 'center', 
                     loc='center')

    # Adjusting column width and font properties
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.auto_set_column_width(col=list(range(len(metrics))))
    for (i, j), cell in table.get_celld().items():
        cell.set_edgecolor("black")
        if i == 0 or j == -1:  # header or row titles
            cell.set_text_props(weight='bold', color='black')
            cell.set_facecolor('#f5f5f5')  # light gray
        else:
            cell.set_facecolor('white')
    
    plt.title(f"{player} Average and Deviation")
    
    # Save the table as an image
    output_file_path = os.path.join(base_dir, player, f"{player} Average and Deviation.png")
    plt.savefig(output_file_path, format='png', dpi=300, bbox_inches='tight')
    
    plt.close()

print("Tables saved for all players in their respective folders!")


KeyError: 'Worksheet Zach Liggett does not exist.'

## Composite score

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Add Players Names based on who you need to add or take out

# Change the name of the excel files to match... look at the text for xlsx documents
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
excel_file = f"{base_dir}/Fall2Winter.xlsx"

player_names = [
    "Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
    "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
    "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
    "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
    "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki", "Ryan Buckner"
]

weights = {
    "Bat Speed (mph)": 1.2,
    "Rotational Acceleration (g)": 1.25,
    "Time to Contact (sec)": 1.05,
    "On Plane Efficiency (%)": 1.15,
    "Early Connection (deg)": 1.1,
    "Connection at Impact (deg)": 1.15
}

all_data = []

# Extract data for each player
for player in player_names:
    try:
        df = pd.read_excel(excel_file, sheet_name=player, skiprows=8, engine='openpyxl')
        avg_data = df[list(weights.keys())].mean().to_dict()
        avg_data["Player"] = player
        all_data.append(avg_data)
    except Exception as e:
        print(f"Error processing {player}: {str(e)}")

# Convert to DataFrame
df_all = pd.DataFrame(all_data)

# Rank each metric
rankings = {}
for metric, weight in weights.items():
    if metric in ["Early Connection (deg)", "Connection at Impact (deg)"]:
        # Rank players based on the absolute difference from 90 degrees
        rankings[metric] = df_all[metric].apply(lambda x: abs(90 - x)).rank()
    else:
        rankings[metric] = df_all[metric].rank(ascending=(metric != "Time to Contact (sec)"))

# Rank each metric
rankings = {}
for metric, weight in weights.items():
    rankings[metric] = df_all[metric].rank(ascending=(metric != "Time to Contact (sec)"))

# Create a DataFrame for rankings
ranking_df = pd.DataFrame(rankings)

# Calculate the composite score
df_all["Composite Score"] = ranking_df.dot(list(weights.values()))
df_all["Composite Ranking"] = df_all["Composite Score"].rank().astype(int)

# Convert rankings to 1/x format
for metric in weights.keys():
    df_all[f"Rank_{metric}"] = ranking_df[metric].astype(int).apply(lambda x: f"{x}/{len(player_names)}")

df_all["Composite Ranking"] = df_all["Composite Ranking"].apply(lambda x: f"{x}/{len(player_names)}")

# Compute the team average for each metric
team_avgs_raw = pd.DataFrame(all_data).mean().to_dict()

# Create and save images
for index, row in df_all.iterrows():
    try:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.axis('tight')
        ax.axis('off')

        # Player's data
        player_data = [row["Composite Ranking"]] + [row[f"Rank_{metric}"] for metric in weights.keys()]

        # Team averages
        team_data = ["Team Avg."] + [f"{team_avgs_raw[metric]:.2f}" for metric in weights.keys()]

        columns = ["Composite Ranking"] + list(weights.keys())
        table_data = [player_data, team_data]

        table = ax.table(cellText=table_data, colLabels=columns, cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.auto_set_column_width(col=list(range(len(columns))))
        for (i, j), cell in table.get_celld().items():
            cell.set_edgecolor("black")
            if i == 0:
                cell.set_text_props(weight='bold', color='black')
                cell.set_facecolor('#f5f5f5')  # light gray
            else:
                cell.set_facecolor('white')

        plt.title(f"{row['Player']} Practice Swings", fontsize=14)

        # Save the table as an image
        player_folder = os.path.join(base_dir, row['Player'])
        if not os.path.exists(player_folder):
            os.makedirs(player_folder)

        output_file_path = os.path.join(player_folder, f"{row['Player']} Rankings.png")
        plt.savefig(output_file_path, format='png', dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Successfully saved rankings for {row['Player']}.")
    except Exception as e:
        print(f"Failed to save rankings for {row['Player']}. Error: {str(e)}")


Successfully saved rankings for Colin Bergmann.
Successfully saved rankings for Brady Kindhart.
Successfully saved rankings for Dominic Sharp.
Successfully saved rankings for Elias Stevens.
Successfully saved rankings for Jordan Rosenberg.
Successfully saved rankings for Matthew Arnold.
Successfully saved rankings for Steven Schnieder.
Successfully saved rankings for Josh Herget.
Successfully saved rankings for Michael Gould.
Successfully saved rankings for Gavin Schacher.
Successfully saved rankings for Michael Long.
Successfully saved rankings for Camden Ruby.
Successfully saved rankings for Jack Zebig.
Successfully saved rankings for Aaris Stolte.
Successfully saved rankings for Andy Bunton.
Successfully saved rankings for Matt Schuler.
Successfully saved rankings for Harry Oden.
Successfully saved rankings for Nick Daugherty.
Successfully saved rankings for Jacob Hager.
Successfully saved rankings for Jordan Stucki.
Successfully saved rankings for Ryan Buckner.


## New Code for Rankings 

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Add Players Names based on who you need to add or take out

# Change the name of the excel files to match... look at the text for xlsx documents
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
excel_file = f"{base_dir}/Fall2Winter.xlsx"

player_names = [
    "Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
    "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
    "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
    "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
    "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki","Ryan Buckner"
]

weights = {
    "Bat Speed (mph)": 1.2,
    "Rotational Acceleration (g)": 1.25,
    "Time to Contact (sec)": 1.05,
    "On Plane Efficiency (%)": 1.15,
    "Early Connection (deg)": 1.1,
    "Connection at Impact (deg)": 1.15
}

all_data = []

# Extract data for each player
for player in player_names:
    try:
        df = pd.read_excel(excel_file, sheet_name=player, skiprows=8, engine='openpyxl')
        avg_data = df[list(weights.keys())].mean().to_dict()
        avg_data["Player"] = player
        all_data.append(avg_data)
    except Exception as e:
        print(f"Error processing {player}: {str(e)}")

# Convert to DataFrame
df_all = pd.DataFrame(all_data)

# Rank each metric
rankings = {}
for metric, weight in weights.items():
    if metric in ["Early Connection (deg)", "Connection at Impact (deg)"]:
        # Rank players based on the absolute difference from 90 degrees, with smaller differences ranked higher
        rankings[metric] = df_all[metric].apply(lambda x: abs(90 - x)).rank(ascending=False)
    else:
        rankings[metric] = df_all[metric].rank(ascending=(metric != "Time to Contact (sec)"))

# Create a DataFrame for rankings
ranking_df = pd.DataFrame(rankings)

# Calculate the composite score
df_all["Composite Score"] = ranking_df.dot(list(weights.values()))
df_all["Composite Ranking"] = df_all["Composite Score"].rank().astype(int)

# Convert rankings to 1/x format
for metric in weights.keys():
    df_all[f"Rank_{metric}"] = ranking_df[metric].astype(int).apply(lambda x: f"{x}/{len(player_names)}")

df_all["Composite Ranking"] = df_all["Composite Ranking"].apply(lambda x: f"{x}/{len(player_names)}")

# Compute the team average for each metric
team_avgs_raw = pd.DataFrame(all_data).mean().to_dict()

# Create and save images
for index, row in df_all.iterrows():
    try:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.axis('tight')
        ax.axis('off')

        # Player's data
        player_data = [row["Composite Ranking"]] + [row[f"Rank_{metric}"] for metric in weights.keys()]

        # Team averages
        team_data = ["Team Avg."] + [f"{team_avgs_raw[metric]:.2f}" for metric in weights.keys()]

        columns = ["Composite Ranking"] + list(weights.keys())
        table_data = [player_data, team_data]

        table = ax.table(cellText=table_data, colLabels=columns, cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.auto_set_column_width(col=list(range(len(columns))))
        for (i, j), cell in table.get_celld().items():
            cell.set_edgecolor("black")
            if i == 0:
                cell.set_text_props(weight='bold', color='black')
                cell.set_facecolor('#f5f5f5')  # light gray
            else:
                cell.set_facecolor('white')

        plt.title(f"{row['Player']} Practice Swings", fontsize=14)

        # Save the table as an image
        player_folder = os.path.join(base_dir, row['Player'])
        if not os.path.exists(player_folder):
            os.makedirs(player_folder)

        output_file_path = os.path.join(player_folder, f"{row['Player']} Rankings.png")
        plt.savefig(output_file_path, format='png', dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Successfully saved rankings for {row['Player']}.")
    except Exception as e:
        print(f"Failed to save rankings for {row['Player']}. Error: {str(e)}")


Successfully saved rankings for Colin Bergmann.
Successfully saved rankings for Brady Kindhart.
Successfully saved rankings for Dominic Sharp.
Successfully saved rankings for Elias Stevens.
Successfully saved rankings for Jordan Rosenberg.
Successfully saved rankings for Matthew Arnold.
Successfully saved rankings for Steven Schnieder.
Successfully saved rankings for Josh Herget.
Successfully saved rankings for Michael Gould.
Successfully saved rankings for Gavin Schacher.
Successfully saved rankings for Michael Long.
Successfully saved rankings for Camden Ruby.
Successfully saved rankings for Jack Zebig.
Successfully saved rankings for Aaris Stolte.
Successfully saved rankings for Andy Bunton.
Successfully saved rankings for Matt Schuler.
Successfully saved rankings for Harry Oden.
Successfully saved rankings for Nick Daugherty.
Successfully saved rankings for Jacob Hager.
Successfully saved rankings for Jordan Stucki.
Successfully saved rankings for Ryan Buckner.


## Distribution graphs

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

# Add Players Names based on who you need to add or take out

# Change the name of the excel files to match... look at the text for xlsx documents
player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Ryan Buckner"]

# Base directory
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"

def plot_kde_frequency_count(df, metric_column, title, ax):
    """Plot KDE frequency distribution as count for the specified metric from the dataframe."""
    
    # Generate KDE plot
    data = df[metric_column].dropna()
    density = gaussian_kde(data)
    xs = np.linspace(data.min(), data.max(), 1000)
    density.covariance_factor = lambda: .25
    density._compute_covariance()
    density_values = density(xs)
    
    # Convert KDE density to count
    count_values = density_values * len(data) * np.diff(xs)[0]
    ax.plot(xs, count_values, color='blue')
    
    # Draw vertical lines for 1/3 and 2/3 quantiles
    lower_third = data.quantile(1/3)
    upper_third = data.quantile(2/3)
    ax.axvline(lower_third, color='red', linestyle='--', linewidth=0.8)
    ax.axvline(upper_third, color='red', linestyle='--', linewidth=0.8)
    
    # Annotate the graph with Average, Q1, and Q2 values
    avg_value = data.mean()
    annotations = [
        (f"Q1: {lower_third:.2f}", lower_third, 10),
        (f"Q2: {upper_third:.2f}", upper_third, 30),
        (f"Average: {avg_value:.2f}", avg_value, 50)
    ]
    for annotation, x_position, vertical_offset in annotations:
        ax.annotate(annotation, xy=(x_position, 0), xytext=(10, vertical_offset), 
                    textcoords='offset points', ha='left', va='bottom', fontsize=8)
    
    # Set title and labels with reduced font size
    ax.set_title(f"{title} Distribution Graph", fontsize=10)
    ax.set_xlabel(metric_column, fontsize=9)
    ax.set_ylabel("Count", fontsize=9)
    ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Adjust aesthetics
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.set_aspect(1.0/ax.get_data_ratio()*2.5/2.5)  # Set the aspect to make graph 2.5x2.5

def generate_and_save_kde_frequency_count_for_player(player_name):
    # Load player data
    df_player = pd.read_excel(f"{base_dir}/Fall2Winter.xlsx", sheet_name=player_name, skiprows=8, engine='openpyxl')
    
    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(1, 3, figsize=(9, 3))

    # Generate plots
    plot_kde_frequency_count(df_player, 'Bat Speed (mph)', player_name, axs[0])
    plot_kde_frequency_count(df_player, 'Attack Angle (deg)', player_name, axs[1])
    plot_kde_frequency_count(df_player, 'Vertical Bat Angle (deg)', player_name, axs[2])

    # Adjust layout
    plt.tight_layout()

    # Save the plot as an image
    output_path = f"{base_dir}/{player_name}/{player_name} Frequency Distribution.png"
    plt.savefig(output_path, format='png', dpi=300)
    
    # Close the plot to free up memory
    plt.close()

# Generate and save plots for each player
for player in player_names:
    generate_and_save_kde_frequency_count_for_player(player)

print("KDE frequency distribution plots (as counts) with 1/3, 2/3 quantiles, and spaced annotations generated and saved for all players!")


KDE frequency distribution plots (as counts) with 1/3, 2/3 quantiles, and spaced annotations generated and saved for all players!


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

# List of player names (sheet names)
player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Ryan Buckner"]

# Base directory
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"

def plot_kde_frequency_count(df, metric_column, title, ax):
    """Plot KDE frequency distribution as count for the specified metric from the dataframe."""
    
    # Generate KDE plot
    data = df[metric_column].dropna()
    density = gaussian_kde(data)
    xs = np.linspace(data.min(), data.max(), 1000)
    density.covariance_factor = lambda: .25
    density._compute_covariance()
    density_values = density(xs)
    
    # Convert KDE density to count
    count_values = density_values * len(data) * np.diff(xs)[0]
    ax.plot(xs, count_values, color='blue')
    
    # Draw vertical lines for 1/3 and 2/3 quantiles
    lower_third = data.quantile(1/3)
    upper_third = data.quantile(2/3)
    ax.axvline(lower_third, color='red', linestyle='--', linewidth=0.8)
    ax.axvline(upper_third, color='red', linestyle='--', linewidth=0.8)
    
    # Annotate the graph with Average, Q1, and Q2 values
    avg_value = data.mean()
    annotations = [
        (f"Q1: {lower_third:.2f}", lower_third, 10),
        (f"Q2: {upper_third:.2f}", upper_third, 30),
        (f"Average: {avg_value:.2f}", avg_value, 50)
    ]
    for annotation, x_position, vertical_offset in annotations:
        ax.annotate(annotation, xy=(x_position, 0), xytext=(10, vertical_offset), 
                    textcoords='offset points', ha='left', va='bottom', fontsize=8)
    
    # Set title and labels with reduced font size
    ax.set_title(f"{title} Distribution Graph", fontsize=10)
    ax.set_xlabel(metric_column, fontsize=9)
    ax.set_ylabel("Count", fontsize=9)
    ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Adjust aesthetics
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.set_aspect(1.0/ax.get_data_ratio()*2.5/2.5)  # Set the aspect to make graph 2.5x2.5

def generate_and_save_kde_frequency_count_for_player(player_name):
    # Load player data
    df_player = pd.read_excel(f"{base_dir}/Fall2Winter.xlsx", sheet_name=player_name, skiprows=8, engine='openpyxl')
    
    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(1, 3, figsize=(9, 3))

    # Generate plots
    plot_kde_frequency_count(df_player, 'Rotational Acceleration (g)', f"{player_name} RA", axs[0])
    plot_kde_frequency_count(df_player, 'Time to Contact (sec)', f"{player_name} TTC", axs[1])
    plot_kde_frequency_count(df_player, 'Peak Hand Speed (mph)', f"{player_name} HS", axs[2])

    # Adjust layout
    plt.tight_layout()

    # Save the plot as an image
    output_path = f"{base_dir}/{player_name}/{player_name} RA TTC HS Distribution.png"
    plt.savefig(output_path, format='png', dpi=300)
    
    # Close the plot to free up memory
    plt.close()

# Generate and save plots for each player
for player in player_names:
    generate_and_save_kde_frequency_count_for_player(player)

print("KDE frequency distribution plots (as counts) for RA, TTC, and HS with 1/3, 2/3 quantiles, and spaced annotations generated and saved for all players!")


KDE frequency distribution plots (as counts) for RA, TTC, and HS with 1/3, 2/3 quantiles, and spaced annotations generated and saved for all players!


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

# List of player names (sheet names)
player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Ryan Buckner"]

# Base directory
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"

def plot_kde_frequency_count(df, metric_column, title, ax):
    """Plot KDE frequency distribution as count for the specified metric from the dataframe."""
    
    # Generate KDE plot
    data = df[metric_column].dropna()
    density = gaussian_kde(data)
    xs = np.linspace(data.min(), data.max(), 1000)
    density.covariance_factor = lambda: .25
    density._compute_covariance()
    density_values = density(xs)
    
    # Convert KDE density to count
    count_values = density_values * len(data) * np.diff(xs)[0]
    ax.plot(xs, count_values, color='blue')
    
    # Draw vertical lines for 1/3 and 2/3 quantiles
    lower_third = data.quantile(1/3)
    upper_third = data.quantile(2/3)
    ax.axvline(lower_third, color='red', linestyle='--', linewidth=0.8)
    ax.axvline(upper_third, color='red', linestyle='--', linewidth=0.8)
    
    # Annotate the graph with Average, Q1, and Q2 values
    avg_value = data.mean()
    annotations = [
        (f"Q1: {lower_third:.2f}", lower_third, 10),
        (f"Q2: {upper_third:.2f}", upper_third, 30),
        (f"Average: {avg_value:.2f}", avg_value, 50)
    ]
    for annotation, x_position, vertical_offset in annotations:
        ax.annotate(annotation, xy=(x_position, 0), xytext=(10, vertical_offset), 
                    textcoords='offset points', ha='left', va='bottom', fontsize=8)
    
    # Set title and labels with reduced font size
    ax.set_title(f"{title}", fontsize=10)
    ax.set_xlabel(metric_column, fontsize=9)
    ax.set_ylabel("Count", fontsize=9)
    ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Adjust aesthetics
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.set_aspect(1.0/ax.get_data_ratio()*2.5/2.5)  # Set the aspect to make graph 2.5x2.5

def generate_and_save_kde_frequency_count_for_player(player_name):
    # Load player data
    df_player = pd.read_excel(f"{base_dir}/Fall2Winter.xlsx", sheet_name=player_name, skiprows=8, engine='openpyxl')
    
    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(1, 3, figsize=(9, 3))

    # Generate plots
    plot_kde_frequency_count(df_player, 'On Plane Efficiency (%)', f"{player_name} OPE", axs[0])
    plot_kde_frequency_count(df_player, 'Early Connection (deg)', f"{player_name} EC", axs[1])
    plot_kde_frequency_count(df_player, 'Connection at Impact (deg)', f"{player_name} CAI", axs[2])

    # Adjust layout
    plt.tight_layout()

    # Save the plot as an image
    output_path = f"{base_dir}/{player_name}/{player_name} OPE EC CAI Distribution.png"
    plt.savefig(output_path, format='png', dpi=300)
    
    # Close the plot to free up memory
    plt.close()

# Generate and save plots for each player
for player in player_names:
    generate_and_save_kde_frequency_count_for_player(player)

print("KDE frequency distribution plots (as counts) for OPE, EC, and CAI with 1/3, 2/3 quantiles, and spaced annotations generated and saved for all players!")


KDE frequency distribution plots (as counts) for OPE, EC, and CAI with 1/3, 2/3 quantiles, and spaced annotations generated and saved for all players!


## Team vs Player Radar chart

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add Players Names based on who you need to add or take out

# Change the name of the excel files to match... look at the text for xlsx documents
player_names = [
    "Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
    "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
    "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
    "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
    "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
    "Ryan Buckner"
]

# Base directory and file path
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
file_path = f'{base_dir}/Fall2Winter.xlsx'

# Initialize an empty DataFrame to store individual averages
player_avg_df = pd.DataFrame()

# Outer limits for scaling to percentages
outer_limits = {
    "BSA": 77,
    "BSM": 88,
    "RAA": 26,
    "RAM": 30,
    "RSA": 80,
    "CSA": 80,
    "PSA": 80
}

# Loop over all players to read their Excel sheet and calculate the metrics
for name in player_names:
    try:
        sheet = pd.read_excel(file_path, sheet_name=name, skiprows=8, engine='openpyxl')
        
        avg_stats = {
            'BSA': sheet['Bat Speed (mph)'].mean(),
            'BSM': sheet['Bat Speed (mph)'].max(),
            'RAA': sheet['Rotational Acceleration (g)'].mean(),
            'RAM': sheet['Rotational Acceleration (g)'].max(),
            'RSA': sheet['Rotation Score'].mean(),
            'CSA': sheet['Connection Score'].mean(),
            'PSA': sheet['Plane Score'].mean()
        }
        avg_stats["Name"] = name
        player_avg_df = player_avg_df.append(avg_stats, ignore_index=True)
    
    except Exception as e:
        print(f"An error occurred while processing the sheet for {name}: {str(e)}")

# Calculate team averages from individual averages
team_avgs = player_avg_df.drop(columns="Name").mean()

# Scale team averages to percentages
scaled_team_avgs = [(team_avgs[label] / outer_limits[label]) * 100 for label in outer_limits.keys()]

# Labels for the radar chart
labels = outer_limits.keys()

# Create radar charts
for index, row in player_avg_df.iterrows():
    scaled_values = [(row[label] / outer_limits[label]) * 100 for label in labels]

    fig, ax = plt.subplots(figsize=(4.5, 4.5), subplot_kw=dict(polar=True))
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    ax.fill(angles, scaled_team_avgs, color='darkgreen', alpha=1, label='Team')
    ax.fill(angles, scaled_values, color='lightgreen', alpha=0.6, label='Individual')
    
    # Add percent difference annotations
    for i, angle in enumerate(angles):
        diff = scaled_values[i] - scaled_team_avgs[i]
        ax.annotate(f"{diff:.2f}%", (angle, scaled_values[i]), textcoords="offset points", xytext=(0,10), ha='center', color='black')

    ax.set_xticks(angles)
    ax.set_xticklabels(labels)
    ax.yaxis.grid(True)
    ax.set_yticklabels([])

    # Adjust legend position to bottom left
    ax.legend(loc='lower left', bbox_to_anchor=(-0.12, -0.12))
    
    plt.title(f"{row['Name']}'s Radar Chart with Team Avg")

    # Save the radar chart as an image in each player's folder
    plt.savefig(f"{base_dir}/{row['Name']}/{row['Name']} Radar Chart.png", format='png', dpi=300)
    
    # Close the plot to free up memory
    plt.close()

print("Radar charts with team averages generated and saved for all players!")


Radar charts with team averages generated and saved for all players!


## Game vs Practice 

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Paths and player names
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
game_swings_path = f"{base_dir}/Game Swings.xlsx"

player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Zach Liggett", "Steven Schnieder L"]

# Metrics list
metrics = ["Plane Score", "Connection Score", "Rotation Score", "Bat Speed (mph)", 
           "Rotational Acceleration (g)", "On Plane Efficiency (%)", "Attack Angle (deg)", 
           "Early Connection (deg)", "Connection at Impact (deg)", "Vertical Bat Angle (deg)", 
           "Power (kW)", "Time to Contact (sec)", "Peak Hand Speed (mph)"]

for player in player_names:
    # Check if player sheet exists in Game Swings.xlsx
    if player in pd.ExcelFile(game_swings_path, engine='openpyxl').sheet_names:
        player_data = pd.read_excel(game_swings_path, sheet_name=player, skiprows=8, engine='openpyxl')
        
        # Calculate the mean and standard deviation for each metric
        means = [round(player_data[metric].mean(), 3) for metric in metrics]
        std_devs = [round(player_data[metric].std(), 3) for metric in metrics]
        
        # Create a table using matplotlib
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=[means, std_devs], 
                         colLabels=metrics, 
                         rowLabels=['Average', 'Standard Deviation'], 
                         cellLoc = 'center', 
                         loc='center')

        # Adjusting column width and font properties
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.auto_set_column_width(col=list(range(len(metrics))))
        for (i, j), cell in table.get_celld().items():
            cell.set_edgecolor("black")
            if i == 0 or j == -1:  # header or row titles
                cell.set_text_props(weight='bold', color='black')
                cell.set_facecolor('#f5f5f5')  # light gray
            else:
                cell.set_facecolor('white')
        
        plt.title(f"{player} Game Swings")
        
        # Save the table as an image
        output_file_path = os.path.join(base_dir, player, f"{player} Game Swings.png")
        plt.savefig(output_file_path, format='png', dpi=300, bbox_inches='tight')
        
        plt.close()
    else:
        print(f"Sheet for {player} not found in Game Swings.xlsx. Skipping this player.")

print("Tables saved for all players in their respective folders!")


Tables saved for all players in their respective folders!


## Game vs Practice 

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Paths and player names
base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"
week5_path = f"{base_dir}/Fall1.xlsx"
game_swings_path = f"{base_dir}/Game Swings.xlsx"

player_names = ["Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
                "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
                "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
                "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
                "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
                "Zach Liggett", "Steven Schnieder L"]

# Outer limits for scaling to percentages
outer_limits = {
    "Bat Speed Avg": 77,
    "Rotational Acceleration Avg": 26,
    "Rotation Score Avg": 80,
    "Connection Score Avg": 80,
    "Plane Score Avg": 80
}

# Function to add data labels
def add_data_labels(ax, values, angles, offset_pct=0, color='black'):
    for i, angle in enumerate(angles):
        distance = (values[i] / outer_limits[list(outer_limits.keys())[i]]) * 100 - (values[i] / outer_limits[list(outer_limits.keys())[i]]) * 100 * offset_pct / 100
        ax.annotate(f"{values[i]:.2f}", (angle, distance), textcoords="offset points", xytext=(0,10), ha='center', fontsize=8, color=color)

# Create radar charts
for player in player_names:
    # Check and create player's directory if it doesn't exist
    player_dir = f"{base_dir}/{player}"
    if not os.path.exists(player_dir):
        os.makedirs(player_dir)

    # Load data from Week5.xlsx
    week5_data = pd.read_excel(week5_path, sheet_name=player, skiprows=8, engine='openpyxl')
    week5_avgs = {
        'Bat Speed Avg': week5_data['Bat Speed (mph)'].mean(),
        'Rotational Acceleration Avg': week5_data['Rotational Acceleration (g)'].mean(),
        'Rotation Score Avg': week5_data['Rotation Score'].mean(),
        'Connection Score Avg': week5_data['Connection Score'].mean(),
        'Plane Score Avg': week5_data['Plane Score'].mean()
    }
    
    # Check if player sheet exists in Game Swings.xlsx
    if player in pd.ExcelFile(game_swings_path, engine='openpyxl').sheet_names:
        game_data = pd.read_excel(game_swings_path, sheet_name=player, skiprows=8, engine='openpyxl')
        game_avgs = {
            'Bat Speed Avg': game_data['Bat Speed (mph)'].mean(),
            'Rotational Acceleration Avg': game_data['Rotational Acceleration (g)'].mean(),
            'Rotation Score Avg': game_data['Rotation Score'].mean(),
            'Connection Score Avg': game_data['Connection Score'].mean(),
            'Plane Score Avg': game_data['Plane Score'].mean()
        }
    else:
        print(f"Sheet for {player} not found in Game Swings.xlsx. Skipping this player.")
        continue
    
    # Scale the data for the radar chart
    week5_scaled = [(week5_avgs[label] / outer_limits[label]) * 100 for label in outer_limits.keys()]
    game_scaled = [(game_avgs[label] / outer_limits[label]) * 100 for label in outer_limits.keys()]
    
    labels = outer_limits.keys()
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, game_scaled, color='blue', alpha=0.4, label='Game')
    ax.fill(angles, week5_scaled, color='darkgreen', alpha=0.6, label='Practice')
    
    # Add data labels for week5 (practice) and game data using actual average values
    add_data_labels(ax, list(week5_avgs.values()), angles, offset_pct=15)
    add_data_labels(ax, list(game_avgs.values()), angles, color='blue')
    
    ax.set_xticks(angles)
    ax.set_xticklabels(labels)
    ax.yaxis.grid(True)
    ax.set_yticklabels([])
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
    
    plt.title(f"{player} Practice vs Game Swing")
    plt.tight_layout()
    
    # Save the radar chart to the player's folder
    output_file_path = f"{player_dir}/{player} Practice vs Game Swing.png"
    plt.savefig(output_file_path, format='png', dpi=300)
    
    # Close the plot to free up memory
    plt.close()

print("Radar charts for all players saved in their respective folders!")


Radar charts for all players saved in their respective folders!


## Word Doc Generater

In [16]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import os

base_dir = "/Users/ivanquackenbush/Documents/Maryville baseball"

player_names = [
    "Colin Bergmann", "Brady Kindhart", "Dominic Sharp", "Elias Stevens",
    "Jordan Rosenberg", "Matthew Arnold", "Steven Schnieder", "Josh Herget",
    "Michael Gould", "Gavin Schacher", "Michael Long", "Camden Ruby",
    "Jack Zebig", "Aaris Stolte", "Andy Bunton", "Matt Schuler",
    "Harry Oden", "Nick Daugherty", "Jacob Hager", "Jordan Stucki",
    "Ryan Buckner"
]

def set_font_style(paragraph, font_name, font_size, bold=False, centered=False):
    run = paragraph.add_run()
    font = run.font
    font.name = font_name
    font.size = Pt(font_size)
    font.bold = bold
    if centered:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    return run

for player in player_names:
    player_dir = os.path.join(base_dir, player)
    if not os.path.exists(player_dir):
        print(f"Directory for {player} not found. Skipping...")
        continue

    doc = Document()
    
    # Header
    section = doc.sections[0]
    header = section.header
    p = header.add_paragraph()
    run = set_font_style(p, 'Roboto', 12, bold=True)
    run.text = "Maryville Baseball"
    p.alignment = WD_ALIGN_PARAGRAPH.RIGHT

    # Title
    p = doc.add_paragraph()
    run = set_font_style(p, 'Roboto', 15, bold=True, centered=True)
    run.text = f"{player} Report"
    
    # Analysis Main Title
    doc.add_paragraph("\n\n")
    p = doc.add_paragraph()
    run = set_font_style(p, 'Roboto', 15, bold=True)
    run.text = "Analysis:"
    
    # Columns: Summary and Needs
    doc.add_paragraph("\n\n")
    table = doc.add_table(rows=1, cols=2)
    for cell in table.columns[0].cells:
        cell.width = Inches(3)
    for cell in table.columns[1].cells:
        cell.width = Inches(3)
    table.cell(0, 0).text = 'Summary:'
    table.cell(0, 1).text = 'Needs:'
    for cell in table.row_cells(0):
        set_font_style(cell.paragraphs[0], 'Roboto', 12, bold=True)
    
    doc.add_paragraph("\n")
    
    # Subtitle: Blast Progress and Analysis
    p = doc.add_paragraph()
    run = set_font_style(p, 'Roboto', 12, bold=True)
    run.text = "Blast Progress and Analysis"
    
    # Add images with captions
    image_files = [
        ("Blast Progress.png", "Swing changes by Week"),
        ("Average and Deviation.png", "Your average scores and how much you will typically deviate from your average"),
        ("Frequency Distribution.png", ""),
        ("RA TTC HS Distribution.png", ""),
        ("OPE EC CAI Distribution.png", "Distribution graphs that show you how often you work in a certain range. The y-axis represents how often the variable is hit. The taller the bell is the more frequently you work in this range. The peak of the bell represents the mode. The middle ⅓ represents the range you typically work in."),
        ("Rankings.png", "Rankings based on where you stand on the team in regards to these metrics. The higher the numerator the better the ranking. The composite score is weighted based on how much value and Relevance the metric has. For example batspeed will have a higher weight on the composite score then handspeed."),
        ("Radar Chart.png", "A visual representation your swing vs. the team average")
    ]
    
    for idx, (image_file, caption_text) in enumerate(image_files):
        file_path = os.path.join(player_dir, f"{player} {image_file}")
        if os.path.exists(file_path):
            doc.add_picture(file_path, width=Inches(6))
            p = doc.add_paragraph()
            run = set_font_style(p, 'Roboto', 9)
            run.text = caption_text
            
            # Reduce space between specified images
            if image_file in ["Frequency Distribution.png", "RA TTC HS Distribution.png"] and idx < len(image_files) - 1:
                doc.add_paragraph("\n")
            else:
                doc.add_paragraph("\n\n")
        else:
            print(f"Skipped missing file: {file_path}")

    # Game Swing Main Title
    doc.add_paragraph("\n")
    p = doc.add_paragraph()
    run = set_font_style(p, 'Roboto', 15, bold=True)
    run.text = "Game Swing"
    
    # Columns: Summary and Needs for Game Swing
    doc.add_paragraph("\n\n")
    table = doc.add_table(rows=1, cols=2)
    for cell in table.columns[0].cells:
        cell.width = Inches(3)
    for cell in table.columns[1].cells:
        cell.width = Inches(3)
    table.cell(0, 0).text = 'Summary:'
    table.cell(0, 1).text = 'Needs:'
    for cell in table.row_cells(0):
        set_font_style(cell.paragraphs[0], 'Roboto', 12, bold=True)
    
    # Game vs Practice Swing Image
    file_path = os.path.join(player_dir, f"{player} Practice vs Game Swing.png")
    if os.path.exists(file_path):
        doc.add_picture(file_path, width=Inches(6))
        p = doc.add_paragraph()
        run = set_font_style(p, 'Roboto', 9, centered=True)
        run.text = "A visual representation your game vs. practice swing"
    else:
        print(f"Skipped missing file: {file_path}")

    # Save the document
    output_file_path = os.path.join(player_dir, f"{player} Report.docx")
    try:
        doc.save(output_file_path)
        print(f"Report for {player} saved successfully!")
    except Exception as e:
        print(f"Error saving report for {player}: {str(e)}")


Skipped missing file: /Users/ivanquackenbush/Documents/Maryville baseball/Colin Bergmann/Colin Bergmann Practice vs Game Swing.png
Report for Colin Bergmann saved successfully!
Skipped missing file: /Users/ivanquackenbush/Documents/Maryville baseball/Brady Kindhart/Brady Kindhart Practice vs Game Swing.png
Report for Brady Kindhart saved successfully!
Skipped missing file: /Users/ivanquackenbush/Documents/Maryville baseball/Dominic Sharp/Dominic Sharp Practice vs Game Swing.png
Report for Dominic Sharp saved successfully!
Skipped missing file: /Users/ivanquackenbush/Documents/Maryville baseball/Elias Stevens/Elias Stevens Practice vs Game Swing.png
Report for Elias Stevens saved successfully!
Skipped missing file: /Users/ivanquackenbush/Documents/Maryville baseball/Jordan Rosenberg/Jordan Rosenberg Practice vs Game Swing.png
Report for Jordan Rosenberg saved successfully!
Skipped missing file: /Users/ivanquackenbush/Documents/Maryville baseball/Matthew Arnold/Matthew Arnold Practice vs